In [1]:
from gpt_4o_mini import Trainer, GPT4Model
from svm_tfidf import SVMClassifier
import os
import dotenv
import requests

dotenv.load_dotenv()

c:\Users\riso\Desktop\Prompt-Classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Models

In [2]:
def get_models() -> list:
    url = f"{os.getenv("PROXY_URL")}/models"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {os.getenv("OPENAI_API_KEY")}"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        models = response.json()
        models = [model["id"] for model in models["data"]]
        return models
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
available_models = get_models()
if available_models:
    print("Available models:", available_models)
else:
    print("Failed to retrieve models")

Available models: ['gpt-4o', 'llama3.1:70b', 'llama3.1:8b', 'o1-mini', 'o1-preview', 'gpt-4', 'gpt-4o-mini', 'gpt-4-turbo']


# GPT-4o-mini

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
proxy_url = os.getenv("PROXY_URL")


gpt_model = GPT4Model(api_key=api_key, proxy_url=proxy_url, domain='law', model_name='gpt-4o-mini', train_size=10, test_size=20)
gpt_model.load_data("../data/open_domain_data.csv", "../data/specific_domain_data.csv")

trainer = Trainer(gpt_model.train_data)
compiled_model = trainer.optimize_model()
accuracy = trainer.test_model(gpt_model.test_data)

print(f"Evaluation results: Accuracy {accuracy * 100:.2f}%")

Train data: 10
Test data: 20
Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 5 candidate sets.


Average Metric: 5 / 10  (50.0): 100%|██████████| 10/10 [00:07<00:00,  1.40it/s]


New best score: 50.0 for seed -3
Scores so far: [50.0]
Best score so far: 50.0


Average Metric: 10 / 10  (100.0): 100%|██████████| 10/10 [00:06<00:00,  1.65it/s]


New best score: 100.0 for seed -2
Scores so far: [50.0, 100.0]
Best score so far: 100.0


...

Scores so far: [50.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0


 20%|██        | 2/10 [00:03<00:12,  1.59s/it]


Bootstrapped 2 full traces after 3 examples in round 0.


Average Metric: 10 / 10  (100.0): 100%|██████████| 10/10 [00:05<00:00,  1.74it/s]


Scores so far: [50.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0
8 candidate programs found.
Evaluation results: Accuracy 95.00%


In [4]:
trainer.save_model(f"../models/{gpt_model.model_name}.json")

[('prog', Predict(StringSignature(prompt -> reasoning, target, label
    instructions='Classify if a text is specific for a domain or not.'
    prompt = Field(annotation=str required=True json_schema_extra={'desc': 'The prompt to classify.', '__dspy_field_type': 'input', 'prefix': 'Prompt:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    target = Field(annotation=str required=True json_schema_extra={'desc': 'The target domain to classify the prompt against.', '__dspy_field_type': 'output', 'prefix': 'Target:'})
    label = Field(annotation=str required=True json_schema_extra={'desc': '1, if the input text belong to domain, 0 otherwise.', '__dspy_field_type': 'output', 'prefix': 'Label:'})
)))]
Model saved to ../models/gpt-4o-mini.json


# SVM-TFIDF

In [ ]:
svm_classifier = SVMClassifier(C=0.1, train_size=10_000, test_size=10_000, seed=22)

# Prepare data (assuming 'dummy_open.csv' and 'dummy_specific.csv' are paths to your data files)
X_train, X_test, y_train, y_test = svm_classifier.prepare_data("../data/open_domain_data.csv", "../data/specific_domain_data.csv")

# Train the model
svm_classifier.train(X_train, y_train)

# Cross-validate
cv_results = svm_classifier.cross_validate_model(X_train, y_train, cv=3)
print("Cross-validation Results:", cv_results)

In [ ]:
# Evaluate on test data
evaluation_results = svm_classifier.evaluate(X_test, y_test)
print("Evaluation Results:", evaluation_results)

In [ ]:
# Save the model
svm_classifier.save_model('svm_model.joblib')

# LSTM